In [1]:
%run -i 'ModelParsing.py'

In [2]:
def checkReactionBalance(model_obj,reaction_num, r_without_formula, atoms_set, excluded_atoms_list):
    temp_dic = {}
    Imbalance_stat = False
    for i in atoms_set:
        temp_dic[i] = 0
    if reaction_num not in r_without_formula:
        r = model.reactions[reaction_num]
        for m in r.pairs:
            for a in list(model.metabolites[m[0]].species.description['formula'].keys()):
                temp_dic[a] = temp_dic[a]+(m[1]*model.metabolites[m[0]].species.description['formula'][a])
    for k in excluded_atoms_list:
        if (k in atoms_set and temp_dic[k]!=0):
            #print(reaction_num)
            return Imbalance_stat, temp_dic
    for j in temp_dic:
        if temp_dic[j]!=0:
            Imbalance_stat = True
    return Imbalance_stat, temp_dic

In [8]:
def trueAtomsSet(model_obj,r_without_formula):
    atoms_set = set()
    for r in model.reactions:
        if r.index not in r_without_formula:
            for m in r.pairs:
                atoms_set = atoms_set.union(model.metabolites[m[0]].species.description['formula'].keys())
    print('List of atoms:',atoms_set)
    excluded_atoms_list = [x for x in input('Would you like to exclude any atom?\nIf yes write thier formula and splited with comma. If no press enter.\n example: if you want to exclude oxygen and hydrogen write O,H.\n').split(',')]
    #for i in remove_list:
        #if i in atoms_set:
            #atoms_set.remove(i)
    #print('Final atoms set',atoms_set)
    return atoms_set,excluded_atoms_list

In [9]:
def totalEBCHeck(model):
    meta_with_formulas = [m.index for m in model.metabolites if len(m.species.description['formula'])!=0 ]
    print('This model contains %d metabolites with assigned chemical formula'%len(meta_with_formulas))
    r_without_formula =[]
    for r in model.reactions:
        for m in r.pairs:
            if m[0] not in meta_with_formulas:
                r_without_formula.append(r.index)
    print('This model contains %d reaction without complete set of chemical formulas'%len(set(r_without_formula)))
    excluded_r = find_biomass_reaction_candidates(model)+find_import_export_reactions(model.fullMatrix)
    r_without_formula = r_without_formula + excluded_r
    im_reaction_set = set()
    if len(meta_with_formulas)!=0:
        atoms_set,excluded_atoms_list = trueAtomsSet(model,r_without_formula)
        for r in model.reactions:
            Imbalance_stat, temp_dic  = checkReactionBalance(model,r.index, r_without_formula,atoms_set,excluded_atoms_list)
            if Imbalance_stat:
                im_reaction_set.add(r.index)
        print('Number of imbalance reactions that we found:',len(im_reaction_set))
    
    return im_reaction_set

Set of all atoms in the following models:

{'Fe', 'Mo', 'e', 'Ca', 'Hg', 'N', 'Cl', 'X', 'C', 'S', 'P', 'Se', 'As', 'F', 'Mg', 'I', 'H', 'Co', 'R', 'Cu', 'Zn', 'K', 'Ni', 'Cd', 'Mn', 'W', 'O', 'Na'}


# 1

Organism: Piscirickettsia salmonis

Version: iPS584

Year:2018

Paper link: https://www.frontiersin.org/articles/10.3389/fmicb.2017.02462/full

Model Link: http://psalmonis.cmm.uchile.cl/models/

In [59]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/Psalmonis_genome_scale_models/iPS584.xml')

0) P. salmonis biomass objective function 1) P. salmonis biomass objective function without ions
Please select the correct string from the list above and enter its number, or -1 if none 
0


In [6]:
model.description

{'ID': 'iPS584',
 'Name': '',
 'Source File': '/home/hzabeti/Desktop/UCSD models/Psalmonis_genome_scale_models/iPS584.xml'}

In [7]:
getSize(model.fullMatrix)

(1114, 1323)

In [8]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Satisfied!


In [61]:
im_reaction_set = totalEBCHeck(model)

This model contains 1114 metabolites with assigned chemical formula
This model contains 0 reaction without complete set of chemical formulas
List of atoms: {'Fe', 'H', 'Co', 'Ca', 'Na', 'X', 'P', 'C', 'e', 'Cd', 'Ni', 'O', 'K', 'Zn', 'Mn', 'N', 'Cl', 'R', 'S', 'As', 'Hg', 'Mg', 'Cu'}
Would you like to exclude any atom?
If yes write thier formula and splited with comma. If no press enter.
 example: if you want to exclude oxygen and hydrogen write O,H.
I,K,M,R,U,X,Y,e
Number of imbalance reactions that we found: 0


In [24]:
print(im_reaction_set)

{991}


In [26]:
atoms_set,excluded_atoms_list = trueAtomsSet(model)
Imbalance_stat, temp_dic = checkReactionBalance(model,991, r_without_formula,atoms_set,excluded_atoms_list)

List of atoms: {'C', 'Co', 'Ni', 'O', 'R', 'S', 'Zn', 'H', 'e', 'P', 'Na', 'Fe', 'Hg', 'Ca', 'X', 'Mg', 'As', 'N', 'Cl', 'K', 'Cu', 'Mn', 'Cd'}
Would you like to exclude any atom?
If yes write thier formula and splited with comma. If no press enter.
 example: if you want to exclude oxygen and hydrogen write O,H.



In [27]:
print(temp_dic)

{'C': Fraction(0, 1), 'Co': 0, 'Ni': 0, 'O': Fraction(0, 1), 'P': Fraction(0, 1), 'R': 0, 'S': 0, 'Zn': 0, 'Hg': 0, 'e': Fraction(-4, 1), 'Na': 0, 'Fe': 0, 'Ca': 0, 'X': 0, 'As': 0, 'H': Fraction(0, 1), 'N': Fraction(0, 1), 'Cl': 0, 'K': 0, 'Cu': 0, 'Mn': 0, 'Cd': 0, 'Mg': 0}


In [28]:
for i in model.reactions[991].pairs:
    print(i[1],model.metabolites[i[0]].species.description['formula'])

-1 {'N': 2, 'C': 128, 'e': 6, 'P': 2, 'O': 51, 'H': 228}
-2 {'N': 1, 'C': 60, 'e': 0, 'P': 1, 'O': 7, 'H': 100}
-2 {'N': 2, 'C': 15, 'e': 2, 'P': 2, 'O': 17, 'H': 22}
-1 {'N': 2, 'C': 15, 'e': 2, 'P': 2, 'O': 17, 'H': 22}
-1 {'N': 5, 'C': 16, 'e': 2, 'P': 2, 'O': 16, 'H': 23}
-2 {'N': 5, 'C': 22, 'e': 2, 'P': 1, 'O': 15, 'H': 32}
1 {'N': 8, 'C': 188, 'e': 0, 'P': 2, 'O': 91, 'H': 328}
2 {'C': 55, 'e': 2, 'P': 1, 'H': 89, 'O': 4}
3 {'N': 2, 'C': 9, 'e': 3, 'P': 2, 'O': 12, 'H': 11}
6 {'H': 1, 'e': -1}
1 {'N': 5, 'C': 10, 'e': 3, 'P': 2, 'O': 11, 'H': 12}
2 {'N': 3, 'C': 9, 'e': 2, 'P': 1, 'O': 8, 'H': 12}


# 2

Organism: Streptomyces clavuligerus	

Version: iLT1021

Year: 2018

Paper link: https://link.springer.com/article/10.1007%2Fs00449-018-1900-9

Model Link: https://static-content.springer.com/esm/art%3A10.1007%2Fs00449-018-1900-9/MediaObjects/449_2018_1900_MOESM2_ESM.xml

In [30]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/12918_2008_242_MOESM5_ESM.XML')

In [31]:
getSize(model.fullMatrix)

(828, 996)

In [5]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Not satisfied!
Optimum solution and the number of all possible free lunch metabolites 2


In [6]:
print(flm_index_list,flm_list)

[315, 335] ['M_PROTON_Periplasm', 'M_PROTON_Cytosol']


In [7]:
ind_flm_index_list,ind_flm_list = model.findIndividuallyProducibleFLM()

Number of individually producible free lunch metabolites 2


In [8]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch(free_lunch_meta_list=[315])

Number of iteration:  0
Number of reactions involved in FL: 5
------------------------------------------------------------
Checking index:  18
This index is in the minimal set 18
Checking index:  201
This index is in the minimal set 201
Checking index:  314
This index is in the minimal set 314
Checking index:  325
This index is in the minimal set 325
Checking index:  542
This index is in the minimal set 542
Minimal free lunch indexes: [18, 201, 314, 325, 542]
------------------ minimal free lunch equation----------------
3/4*(1 M_WATER_Cytosol[Cytosol] + 1 M_ATP_Cytosol[Cytosol] -> 1 M_PI_Cytosol[Cytosol] + 1 M_PROTON_Cytosol[Cytosol] + 1 M_ADP_Cytosol[Cytosol])
-1/2*(1 M_WATER_Cytosol[Cytosol] + 1 M_ATP_Cytosol[Cytosol] -> 1 M_AMP_Cytosol[Cytosol] + 1 M_PPI_Cytosol[Cytosol])
-1/4*(1 M_PI_Cytosol[Cytosol] + 4 M_PROTON_Periplasm[Periplasm] + 1 M_ADP_Cytosol[Cytosol] <-> 3 M_PROTON_Cytosol[Cytosol] + 1 M_WATER_Cytosol[Cytosol] + 1 M_ATP_Cytosol[Cytosol])
-1/2*(1 M_PPI_Cytosol[Cytosol] + 

In [9]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 230.000000 after 0.00 sec. (0.16 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 312 rows and 279 columns.
MIP Presolve modified 1245 coefficients.
Aggregator did 18 substitutions.
Reduced MIP has 1418 rows, 1405 columns, and 7203 nonzeros.
Reduced MIP has 797 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (6.20 ticks)
Probing time = 0.00 sec. (0.17 ticks)
Tried aggregator 1 time.
Reduced MIP has 1418 rows, 1405 columns, and 7203 nonzeros.
Reduced MIP has 797 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (2.91 ticks)
Probing time = 0.00 sec. (0.17 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, us

In [38]:
im_reaction_set = totalEBCHeck(model)

This model contains 0 metabolites with assigned chemical formula
This model contains 996 reaction without complete set of chemical formulas


# 3

Organism: Piscirickettsia salmonis

Version: iPF215

Year: 2016

Paper link: https://www.sciencedirect.com/science/article/pii/S0960852416314304?via%3Dihub

Model Link: https://ars.els-cdn.com/content/image/1-s2.0-S0960852416314304-mmc2.xml

In [39]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/1-s2.0-S0960852416314304-mmc2.xml')

In [40]:
getSize(model.fullMatrix)

(449, 476)

In [13]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Not satisfied!
Optimum solution and the number of all possible free lunch metabolites 3


In [14]:
print(flm_index_list,flm_list)

[152, 375, 446] ['M_cpd00067_e0_c', 'M_cpd00067_c0_c', 'M_cpd00067_e0_c_b']


In [15]:
ind_flm_index_list,ind_flm_list = model.findIndividuallyProducibleFLM()

Number of individually producible free lunch metabolites 3


In [16]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch(free_lunch_meta_list=[152])

Number of iteration:  0
Number of reactions involved in FL: 4
------------------------------------------------------------
Checking index:  49
This index is in the minimal set 49
Checking index:  106
This index is in the minimal set 106
Checking index:  302
This index is in the minimal set 302
Checking index:  329
This index is in the minimal set 329
Minimal free lunch indexes: [49, 106, 302, 329]
------------------ minimal free lunch equation----------------
-1*(1 M_cpd00072_c0_c[c] + 1 M_cpd00002_c0_c[c] -> 1 M_cpd00008_c0_c[c] + 1 M_cpd00290_c0_c[c])
3/4*(1 M_cpd00002_c0_c[c] + 1 M_cpd00001_c0_c[c] -> 1 M_cpd00008_c0_c[c] + 1 M_cpd00009_c0_c[c] + 1 M_cpd00067_c0_c[c])
-1*(1 M_cpd00290_c0_c[c] + 1 M_cpd00001_c0_c[c] -> 1 M_cpd00072_c0_c[c] + 1 M_cpd00009_c0_c[c])
-1/4*(1 M_cpd00008_c0_c[c] + 1 M_cpd00009_c0_c[c] + 4 M_cpd00067_e0_c[c] -> 1 M_cpd00002_c0_c[c] + 1 M_cpd00001_c0_c[c] + 3 M_cpd00067_c0_c[c])
-----------
1*(M_cpd00067_e0_c)
------------------------------------------------

In [17]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 204.000000 after 0.00 sec. (0.09 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 162 rows and 165 columns.
MIP Presolve modified 442 coefficients.
Aggregator did 4 substitutions.
Reduced MIP has 784 rows, 755 columns, and 4269 nonzeros.
Reduced MIP has 408 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (3.60 ticks)
Probing time = 0.00 sec. (0.08 ticks)
Tried aggregator 1 time.
Reduced MIP has 784 rows, 755 columns, and 4269 nonzeros.
Reduced MIP has 408 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.68 ticks)
Probing time = 0.00 sec. (0.08 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up

In [41]:
im_reaction_set = totalEBCHeck(model)

This model contains 0 metabolites with assigned chemical formula
This model contains 476 reaction without complete set of chemical formulas


# 4

Organism: Ralstonia solanacearum

Version: iRP1476

Year: 2016

Paper link: https://journals.plos.org/plospathogens/article?id=10.1371/journal.ppat.1005939

Model Link: http://lipm-bioinfo.toulouse.inra.fr/systemsbiology/models/rsolanacearum/data/metabolism/sbml/1.0/iRP1476.xml

In [42]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/iRP1476.xml')

In [43]:
model.description

{'ID': 'iRP1476',
 'Name': 'Ralstonia solanacearum GMI1000 iRP1476',
 'Source File': '/home/hzabeti/Desktop/UCSD models/iRP1476.xml'}

In [43]:
getSize(model.fullMatrix)

(2574, 2644)

In [44]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Satisfied!


In [45]:
ind_flm_index_list,ind_flm_list = model.findIndividuallyProducibleFLM()

Number of individually producible free lunch metabolites 0


In [46]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch()

In [48]:
print(minimal_free_lunch)

[]


In [47]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 1224.000000 after 0.06 sec. (0.59 ticks)
Tried aggregator 3 times.
MIP Presolve eliminated 2176 rows and 2089 columns.
MIP Presolve modified 9076 coefficients.
Aggregator did 17 substitutions.
Reduced MIP has 3093 rows, 3109 columns, and 21209 nonzeros.
Reduced MIP has 1842 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.09 sec. (50.91 ticks)
Probing time = 0.00 sec. (0.26 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 8 rows and 8 columns.
Reduced MIP has 3085 rows, 3101 columns, and 21185 nonzeros.
Reduced MIP has 1838 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (7.88 ticks)
Probing time = 0.00 sec. (0.26 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search 

In [44]:
im_reaction_set = totalEBCHeck(model)

This model contains 0 metabolites with assigned chemical formula
This model contains 2644 reaction without complete set of chemical formulas


# 5

Organism: Mycobacterium tuberculosis

Version: iNJ661m

Year: 2010

Paper link: https://bmcsystbiol.biomedcentral.com/articles/10.1186/1752-0509-4-160

Model Link: https://static-content.springer.com/esm/art%3A10.1186%2F1752-0509-4-160/MediaObjects/12918_2010_571_MOESM2_ESM.XML

In [45]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/12918_2010_571_MOESM2_ESM.XML')

In [46]:
model.description

{'ID': 'iNJ661m_SBML_xml',
 'Name': 'iNJ661m_SBML_xml',
 'Source File': '/home/hzabeti/Desktop/UCSD models/12918_2010_571_MOESM2_ESM.XML'}

In [233]:
getSize(model.fullMatrix)

(934, 1049)

In [34]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Not satisfied!
Optimum solution and the number of all possible free lunch metabolites 242


In [37]:
print(flm_index_list,flm_list)

60 M_4h2opntn_c


In [36]:
ind_flm_index_list,ind_flm_list = model.findIndividuallyProducibleFLM()

Number of individually producible free lunch metabolites 242


In [38]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch(free_lunch_meta_list=[60])

Number of iteration:  0
Number of reactions involved in FL: 31
------------------------------------------------------------
Checking index:  11
This index is in the minimal set 11
Checking index:  24
This index is in the minimal set 24
Checking index:  167
This index is in the minimal set 167
Checking index:  357
This index is in the minimal set 357
Checking index:  359
This index is in the minimal set 359
Checking index:  360
This index is in the minimal set 360
Checking index:  418
This index is in the minimal set 418
Checking index:  428
This index is in the minimal set 428
Checking index:  534
This index is in the minimal set 534
Checking index:  547
This index is in the minimal set 547
Checking index:  584
This index is in the minimal set 584
Checking index:  598
This index is in the minimal set 598
Checking index:  599
This index is in the minimal set 599
Checking index:  601
This index is in the minimal set 601
Checking index:  602
This index is in the minimal set 602
Checking i

In [15]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 508.000000 after 0.07 sec. (0.20 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 486 rows and 477 columns.
MIP Presolve modified 1583 coefficients.
Aggregator did 5 substitutions.
Reduced MIP has 1605 rows, 1499 columns, and 9448 nonzeros.
Reduced MIP has 884 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.15 sec. (9.21 ticks)
Probing time = 0.00 sec. (0.18 ticks)
Tried aggregator 1 time.
Reduced MIP has 1605 rows, 1499 columns, and 9448 nonzeros.
Reduced MIP has 884 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.21 sec. (3.53 ticks)
Probing time = 0.00 sec. (0.18 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, usi

In [52]:
im_reaction_set = totalEBCHeck(model)

This model contains 838 metabolites with assigned chemical formula
This model contains 96 reaction without complete set of chemical formulas
List of atoms: {'C', 'Co', 'Y', 'O', 'R', 'U', 'S', 'H', 'Mg', 'P', 'Na', 'Fe', 'Se', 'Ca', 'X', 'N', 'Cl', 'K', 'M', 'Cu'}
Would you like to exclude any atom?
If yes write thier formula and splited with comma. If no press enter.
 example: if you want to exclude oxygen and hydrogen write O,H.
I,K,M,R,U,X,Y
Number of imbalance reactions that we found: 7


In [53]:
print(im_reaction_set)

{224, 876, 13, 1005, 1006, 953, 223}


# 6

Organism: Homo sapiens 

Version: Recon 2.2 

Year: 2016

Paper link: https://link.springer.com/article/10.1007%2Fs11306-016-1051-4

Model Link: https://github.com/mcisb/mcisb-recon-analysis/tree/master/models

In [25]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/recon_2.2_edited.xml')

0) Generic human biomass reaction 1) protein component of biomass 2) DNA component of biomass 3) RNA component of biomass 4) carbohydrate component of biomass 5) lipid component of biomass 6) other component of biomass
Please select the correct string from the list above and enter its number, or -1 if none 
0


In [28]:
model.description

{'ID': 'recon2_2',
 'Name': 'Recon 2.2',
 'Source File': '/home/hzabeti/Desktop/UCSD models/recon_2.2_edited.xml'}

In [9]:
getSize(model.fullMatrix)

(6047, 7785)

In [243]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Satisfied!


In [27]:
im_reaction_set = totalEBCHeck(model)

This model contains 6047 metabolites with assigned chemical formula
This model contains 0 reaction without complete set of chemical formulas
List of atoms: {'Fe', 'H', 'Co', 'Ca', 'Na', 'X', 'P', 'C', 'Se', 'e', 'O', 'K', 'F', 'Zn', 'N', 'Cl', 'R', 'I', 'S'}
Would you like to exclude any atom?
If yes write thier formula and splited with comma. If no press enter.
 example: if you want to exclude oxygen and hydrogen write O,H.
I,K,M,R,U,X,Y
Number of imbalance reactions that we found: 0


# 7

Organism: Mucor circinelloides

Version: iWV1213 

Year: 2016

Paper link: https://www.sciencedirect.com/science/article/pii/S0378111916300956?via%3Dihub

Model Link: https://ars.els-cdn.com/content/image/1-s2.0-S0378111916300956-mmc4.xml

In [57]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/iWV1213.xml')

In [58]:
model.description

{'ID': '',
 'Name': '',
 'Source File': '/home/hzabeti/Desktop/UCSD models/iWV1213.xml'}

In [26]:
getSize(model.fullMatrix)

(1413, 1326)

In [27]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Not satisfied!
Optimum solution and the number of all possible free lunch metabolites 613


In [ ]:
ind_flm_index_list,ind_flm_list = model.findIndividuallyProducibleFLM()

In [ ]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch()

In [63]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 312.000000 after 0.05 sec. (0.24 ticks)
Tried aggregator 5 times.
MIP Presolve eliminated 903 rows and 1105 columns.
MIP Presolve modified 2657 coefficients.
Aggregator did 26 substitutions.
Reduced MIP has 1639 rows, 1565 columns, and 7964 nonzeros.
Reduced MIP has 923 binaries, 11 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.06 sec. (11.40 ticks)
Probing time = 0.00 sec. (0.17 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 7 columns.
Reduced MIP has 1635 rows, 1558 columns, and 7952 nonzeros.
Reduced MIP has 920 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (3.28 ticks)
Probing time = 0.00 sec. (0.17 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method

In [59]:
im_reaction_set = totalEBCHeck(model)

This model contains 1413 metabolites with assigned chemical formula
This model contains 0 reaction without complete set of chemical formulas
List of atoms: {'e'}
Would you like to exclude any atom?
If yes write thier formula and splited with comma. If no press enter.
 example: if you want to exclude oxygen and hydrogen write O,H.

Number of imbalance reactions that we found: 0


# 8

Organism: Zymomonas mobilis

Version: iEM439 

Year: 2016

Paper link: https://pubs.rsc.org/en/Content/ArticleLanding/2016/MB/C5MB00588D#!divAbstract

Model Link: http://www.rsc.org/suppdata/c5/mb/c5mb00588d/c5mb00588d2.xml

In [29]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/iEM439.xml')

In [65]:
model.description

{'ID': '',
 'Name': '',
 'Source File': '/home/hzabeti/Desktop/UCSD models/iEM439.xml'}

In [66]:
getSize(model.fullMatrix)

(705, 767)

In [7]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Not satisfied!
Optimum solution and the number of all possible free lunch metabolites 1


In [8]:
print(flm_index_list,flm_list)

[531] ['M_apoACP_c']


In [9]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch(free_lunch_meta_list=[531])

Number of iteration:  0
Number of reactions involved in FL: 7
------------------------------------------------------------
Checking index:  123
This index is in the minimal set 123
Checking index:  302
This index is in the minimal set 302
Checking index:  337
This index is in the minimal set 337
Checking index:  561
This index is in the minimal set 561
Checking index:  562
This index is in the minimal set 562
Checking index:  563
This index is in the minimal set 563
Checking index:  564
This index is in the minimal set 564
Minimal free lunch indexes: [123, 302, 337, 561, 562, 563, 564]
------------------ minimal free lunch equation----------------
1*(1 M_h2o_c[c] + 1 M_pap_c[c] -> 1 M_pi_c[c] + 1 M_amp_c[c])
-1*(1 M_atp_c[c] + 1 M_h2o_c[c] -> 1 M_h_c[c] + 1 M_amp_c[c] + 1 M_ppi_c[c])
-1*(1 M_atp_c[c] + 1 M_h2o_c[c] -> 1 M_adp_c[c] + 1 M_h_c[c] + 1 M_pi_c[c])
1*(1 M_atp_c[c] + 1 M_h_c[c] + 1 M_pan4p_c[c] -> 1 M_ppi_c[c] + 1 M_dpcoa_c[c])
1*(1 M_atp_c[c] + 1 M_dpcoa_c[c] -> 1 M_adp_c[c] 

In [10]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 412.000000 after 0.05 sec. (0.15 ticks)
Tried aggregator 3 times.
MIP Presolve eliminated 247 rows and 243 columns.
MIP Presolve modified 1005 coefficients.
Aggregator did 9 substitutions.
Reduced MIP has 1208 rows, 1185 columns, and 6760 nonzeros.
Reduced MIP has 660 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (6.68 ticks)
Probing time = 0.00 sec. (0.13 ticks)
Tried aggregator 1 time.
Reduced MIP has 1208 rows, 1185 columns, and 6760 nonzeros.
Reduced MIP has 660 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (2.62 ticks)
Probing time = 0.00 sec. (0.13 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, usi

In [44]:
im_reaction_set = totalEBCHeck(model)

This model contains 690 metabolites with assigned chemical formula
This model contains 15 reaction without complete set of chemical formulas
List of atoms: {'Fe', 'H', 'N', 'Na', 'X', 'P', 'C', 'R', 'S', 'O', 'K', 'Mg', 'Cu'}
Would you like to exclude any atom?
If yes write thier formula and splited with comma. If no press enter.
 example: if you want to exclude oxygen and hydrogen write O,H.
I,K,M,R,U,X,Y
Number of imbalance reactions that we found: 22


In [45]:
print(sorted(list(im_reaction_set)))

[54, 62, 126, 127, 132, 137, 142, 147, 154, 159, 165, 171, 177, 183, 189, 544, 549, 567, 571, 580, 633, 666]


# 9

Organism:Caenorhabditis elegans
    
Version:iCEL1273	
    
Year: 2016
    
Paper link: https://www.sciencedirect.com/science/article/pii/S240547121630120X?via%3Dihub
    
Model Link:http://wormflux.umassmed.edu/Downloads/iCEL1273.xml

In [69]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/iCEL1273.xml')

0) Drain of Biomass 1) Assembly of minor components of biomass 2) Assembly of body biomass (with storage) 3) Assembly of body biomass (without storage) 4) Assembly of embryonic biomass 5) Assembly of progeny biomass
Please select the correct string from the list above and enter its number, or -1 if none 
0


In [12]:
model.description

{'ID': 'iCEL1273',
 'Name': 'Caenorhabditis_elegans_metabolic_network_model_iCEL1273',
 'Source File': '/home/hzabeti/Desktop/UCSD models/iCEL1273.xml'}

In [13]:
getSize(model.fullMatrix)

(1718, 1985)

In [14]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Not satisfied!
Optimum solution and the number of all possible free lunch metabolites 1297


In [15]:
ind_flm_index_list,ind_flm_list = model.findIndividuallyProducibleFLM()

Number of individually producible free lunch metabolites 160


In [17]:
print(ind_flm_index_list,ind_flm_list)

[5, 8, 13, 24, 30, 33, 37, 38, 41, 62, 63, 73, 76, 79, 80, 81, 90, 98, 99, 311, 317, 318, 322, 463, 464, 465, 466, 467, 468, 479, 480, 513, 517, 533, 534, 535, 536, 537, 538, 539, 540, 584, 585, 588, 590, 591, 593, 595, 596, 597, 598, 599, 600, 622, 623, 645, 647, 792, 794, 796, 797, 807, 819, 821, 927, 928, 931, 966, 968, 970, 972, 1017, 1028, 1062, 1063, 1066, 1079, 1080, 1081, 1099, 1105, 1118, 1133, 1144, 1145, 1149, 1150, 1173, 1193, 1228, 1240, 1241, 1244, 1248, 1257, 1259, 1261, 1265, 1280, 1301, 1320, 1325, 1326, 1327, 1330, 1333, 1348, 1354, 1363, 1374, 1385, 1387, 1389, 1399, 1413, 1427, 1429, 1441, 1449, 1456, 1460, 1466, 1476, 1478, 1482, 1505, 1509, 1528, 1530, 1536, 1540, 1543, 1544, 1545, 1548, 1557, 1559, 1560, 1567, 1570, 1572, 1578, 1584, 1589, 1594, 1597, 1610, 1613, 1616, 1618, 1625, 1649, 1657, 1659, 1664, 1672, 1673, 1675, 1687, 1688] ['C00221', 'C00267', 'C00001', 'C00011', 'M00011', 'C00084', 'M00084', 'M00001', 'C00469', 'M00399', 'M00390', 'C00121', 'C01801', 

In [18]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch(free_lunch_meta_list=[792])

Number of iteration:  0
Number of reactions involved in FL: 2
------------------------------------------------------------
Checking index:  757
This index is in the minimal set 757
Checking index:  1182
This index is in the minimal set 1182
Minimal free lunch indexes: [757, 1182]
------------------ minimal free lunch equation----------------
-1*(1 M02248[Mitochondria] + 1 M00032[Mitochondria] <-> 1 M00126[Mitochondria])
-1*(1 M00126[Mitochondria] -> 1 M02248[Mitochondria])
-----------
1*(M00032)
-------------------------------------------------------------


In [4]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 619.000000 after 0.05 sec. (0.35 ticks)
Tried aggregator 4 times.
MIP Presolve eliminated 1282 rows and 1268 columns.
MIP Presolve modified 2917 coefficients.
Aggregator did 49 substitutions.
Reduced MIP has 2627 rows, 2379 columns, and 14323 nonzeros.
Reduced MIP has 1399 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.07 sec. (18.85 ticks)
Probing time = 0.00 sec. (0.20 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 0 columns.
Reduced MIP has 2626 rows, 2378 columns, and 14317 nonzeros.
Reduced MIP has 1398 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.04 sec. (5.78 ticks)
Probing time = 0.00 sec. (0.20 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search m

In [70]:
im_reaction_set = totalEBCHeck(model)

This model contains 0 metabolites with assigned chemical formula
This model contains 1985 reaction without complete set of chemical formulas


# 10

Organism: Pichia pastoris

Version: iMT1026

Year: 2016

Paper link: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0148031

Model Link: https://journals.plos.org/plosone/article/file?type=supplementary&id=info:doi/10.1371/journal.pone.0148031.s006

In [54]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/iMT1026.XML')

0) Biomass composition (g/g) 1) Biomass exchange 2) Biomass composition 80:20 (g/g) 3) Biomass composition 60:40 (g/g) 4) Biomass composition 40:60 (g/g) 5) Biomass composition 80:20 (g/g) 6) Biomass composition 60:40 (g/g) 7) Biomass composition 40:60 (g/g)
Please select the correct string from the list above and enter its number, or -1 if none 
0


In [55]:
model.description

{'ID': 'iMT1026',
 'Name': 'Pichia pastoris GEM model',
 'Source File': '/home/hzabeti/Desktop/UCSD models/iMT1026.XML'}

In [56]:
getSize(model.fullMatrix)

(2755, 2210)

In [57]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Not satisfied!
Optimum solution and the number of all possible free lunch metabolites 1295


In [16]:
ind_flm_index_list,ind_flm_list = model.findIndividuallyProducibleFLM()

Number of individually producible free lunch metabolites 1293


In [17]:
print(ind_flm_index_list,ind_flm_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 34, 36, 38, 39, 40, 41, 42, 43, 46, 47, 48, 49, 50, 51, 52, 53, 55, 57, 58, 59, 60, 61, 62, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 125, 127, 128, 129, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 181, 182, 183, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238

In [18]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch(free_lunch_meta_list=[5])

Number of iteration:  0
Number of reactions involved in FL: 6
------------------------------------------------------------
Checking index:  22
This index is not in the minimal 22
Checking index:  71
This index is in the minimal set 71
Checking index:  802
This index is in the minimal set 802
Checking index:  1104
This index is in the minimal set 1104
Checking index:  1424
This index is in the minimal set 1424
Checking index:  1924
This index is in the minimal set 1924
Minimal free lunch indexes: [71, 802, 1104, 1424, 1924]
------------------ minimal free lunch equation----------------
-1/4*(1 M_oligoP_v[C_v] + 4 M_h2o_v[C_v] <-> 5 M_pi_v[C_v])
-1*(1 M_h_v[C_v] + 1 M_ile_L_c[C_c] -> 1 M_h_c[C_c] + 1 M_ile_L_v[C_v])
-1/5*(5 M_pi_v[C_v] + 5 M_atp_v[C_v] <-> 1 M_oligoP_v[C_v] + 5 M_adp_v[C_v])
-1*(3 M_h_c[C_c] + 1 M_pi_v[C_v] + 1 M_adp_v[C_v] -> 1 M_h2o_v[C_v] + 2 M_h_v[C_v] + 1 M_atp_v[C_v])
-1*(1 M_h_v[C_v] + 1 M_ile_L_v[C_v] -> 1 M_h_c[C_c] + 1 M_ile_L_c[C_c])
-----------
1/20*(M_oligoP

In [19]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 929.000000 after 0.07 sec. (0.41 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 644 rows and 1794 columns.
MIP Presolve modified 1997 coefficients.
Aggregator did 12 substitutions.
Reduced MIP has 3400 rows, 2977 columns, and 18239 nonzeros.
Reduced MIP has 1780 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.13 sec. (15.79 ticks)
Probing time = 0.00 sec. (0.25 ticks)
Tried aggregator 1 time.
Reduced MIP has 3400 rows, 2977 columns, and 18239 nonzeros.
Reduced MIP has 1780 binaries, 10 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.04 sec. (6.94 ticks)
Probing time = 0.00 sec. (0.25 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: determini

  56923 32042       10.0000    27       11.0000        9.0000 12991155   18.18%
  57357 32041        cutoff             11.0000        9.0000 13009930   18.18%
Elapsed time = 642.95 sec. (324994.20 ticks, tree = 750.77 MB, solutions = 9)
  57737 32031        cutoff             11.0000        9.0000 13241543   18.18%
  58381 31921        cutoff             11.0000        9.0000 13426369   18.18%
  59019 31683        cutoff             11.0000        9.0000 13652981   18.18%
  59779 31550        cutoff             11.0000        9.0000 13827172   18.18%
  60323 31293        cutoff             11.0000        9.0000 13963844   18.18%
  60727 31008        cutoff             11.0000        9.0000 14197913   18.18%
  61622 30584        cutoff             11.0000        9.0000 14367160   18.18%
  62717 30294        cutoff             11.0000        9.0000 14548011   18.18%
  63936 29709        cutoff             11.0000        9.0000 14680584   18.18%
  65275 28397        cutoff             11

In [73]:
im_reaction_set = totalEBCHeck(model)

This model contains 1552 metabolites with assigned chemical formula
This model contains 157 reaction without complete set of chemical formulas
List of atoms: {'H', 'C', 'P', 'Na', 'Fe', 'X', 'O', 'R', 'N', 'S', 'K'}
Would you like to exclude any atom?
If yes write thier formula and splited with comma. If no press enter.
 example: if you want to exclude oxygen and hydrogen write O,H.
X,R,K
Number of imbalance reactions that we found: 31


In [75]:
im = list(im_reaction_set)
FLW = [54, 779, 1104, 1137, 1175, 1183, 1186, 1348, 1390, 1749, 2182]
print([x for x in FLW if x in im])

[1104, 1186]


# 11

Organism:Streptomyces coelicolor

Version:iMK1208

Year:2014

Paper link: https://onlinelibrary.wiley.com/doi/full/10.1002/biot.201300539

Model Link: https://onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fbiot.201300539&file=biot_201300539_sm_suppinfo2.zip

In [64]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/biot_201300539_sm_suppinfo2.xml')

In [6]:
model.description

{'ID': '',
 'Name': '',
 'Source File': '/home/hzabeti/Desktop/UCSD models/biot_201300539_sm_suppinfo2.xml'}

In [99]:
getSize(model.fullMatrix)

(1436, 1859)

In [100]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Satisfied!


In [65]:
im_reaction_set = totalEBCHeck(model)

This model contains 1436 metabolites with assigned chemical formula
This model contains 0 reaction without complete set of chemical formulas
List of atoms: {'Fe', 'H', 'Co', 'Ca', 'Na', 'X', 'P', 'C', 'e', 'Cd', 'Ni', 'O', 'K', 'Zn', 'Mn', 'N', 'Cl', 'R', 'W', 'S', 'As', 'Mo', 'Mg', 'Cu'}
Would you like to exclude any atom?
If yes write thier formula and splited with comma. If no press enter.
 example: if you want to exclude oxygen and hydrogen write O,H.
I,K,M,R,U,X,Y,e
Number of imbalance reactions that we found: 2


In [66]:
print(im_reaction_set)

{1531, 1543}


# 12

Organism:Clostridium difficile

Version: iMLTC806cdf

Year: 2014

Paper link: https://bmcsystbiol.biomedcentral.com/articles/10.1186/s12918-014-0117-z

Model Link: https://static-content.springer.com/esm/art%3A10.1186%2Fs12918-014-0117-z/MediaObjects/12918_2014_117_MOESM1_ESM.zip

In [13]:
model =parseSBML('/home/hzabeti/Desktop/UCSD models/iMLTC806cdf.xml')

0) Biomass Reaction 1) Exchange Biomass
Please select the correct string from the list above and enter its number, or -1 if none 
0


In [14]:
model.description

{'ID': 'iMLTC806cdf',
 'Name': 'Clostridium difficile strain 630 model by  Mathieu Larocque and Thierry Chenard under the supervision of Rafael Najmanovich',
 'Source File': '/home/hzabeti/Desktop/UCSD models/iMLTC806cdf.xml'}

In [15]:
getSize(model.fullMatrix)

(1756, 1219)

In [16]:
flm_index_list,flm_list = model.findFreeLunchMetabolites()

No Free Lunch condition: Not satisfied!
Optimum solution and the number of all possible free lunch metabolites 17


In [17]:
ind_flm_index_list,ind_flm_list = model.findIndividuallyProducibleFLM()

Number of individually producible free lunch metabolites 17


In [18]:
minimal_free_lunch,free_lunch_list,free_lunch_meta = model.findMinimalFreeLunch()

Number of iteration:  0
Number of reactions involved in FL: 4
------------------------------------------------------------
Checking index:  401
This index is in the minimal set 401
Checking index:  402
This index is in the minimal set 402
Checking index:  433
This index is in the minimal set 433
Checking index:  435
This index is in the minimal set 435
Minimal free lunch indexes: [401, 402, 433, 435]
------------------ minimal free lunch equation----------------
1*(1 M_h_c[C_c] + 1 M_nadph_c[C_c] + 1 M_4s4h2345th2sdl_c[C_c] -> 1 M_h2o_c[C_c] + 1 M_nadp_c[C_c] + 1 M_2345thd_c[C_c])
-1*(1 M_nadh_c[C_c] + 1 M_h_c[C_c] + 1 M_4s4h2345th2sdl_c[C_c] -> 1 M_nad_c[C_c] + 1 M_2345thd_c[C_c])
-1/3*(3 M_h_c[C_c] + 3 M_nadph_c[C_c] + 1 M_no2_c[C_c] -> 2 M_h2o_c[C_c] + 1 M_nh3_c[C_c] + 3 M_nadp_c[C_c])
1/3*(3 M_nadh_c[C_c] + 3 M_h_c[C_c] + 1 M_no2_c[C_c] -> 2 M_h2o_c[C_c] + 3 M_nad_c[C_c] + 1 M_nh3_c[C_c])
-----------
1*(M_h2o_c)
-------------------------------------------------------------


In [19]:
flw_index_list, y_lower_bound = model.findFreeLunchWitnesses()

parameters.mip.tolerances.integrality
CPXPARAM_Simplex_Tolerances_Feasibility          1.0000000000000001e-09
CPXPARAM_Read_DataCheck                          1
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Found incumbent of value 409.000000 after 0.00 sec. (0.24 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 521 rows and 1312 columns.
MIP Presolve modified 1164 coefficients.
Aggregator did 9 substitutions.
Reduced MIP has 1904 rows, 1652 columns, and 9522 nonzeros.
Reduced MIP has 987 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (8.44 ticks)
Probing time = 0.00 sec. (0.20 ticks)
Tried aggregator 1 time.
Reduced MIP has 1904 rows, 1652 columns, and 9522 nonzeros.
Reduced MIP has 987 binaries, 7 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (3.73 ticks)
Probing time = 0.00 sec. (0.20 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, us

In [20]:
im_reaction_set = totalEBCHeck(model)

This model contains 793 metabolites with assigned chemical formula
This model contains 178 reaction without complete set of chemical formulas
List of atoms: {'Zn', 'Fe', 'H', 'Co', 'Ca', 'N', 'Na', 'P', 'C', 'R', 'S', 'O', 'K', 'Mg'}
Would you like to exclude any atom?
If yes write thier formula and splited with comma. If no press enter.
 example: if you want to exclude oxygen and hydrogen write O,H.
I,K,M,R,U,X,Y
Number of imbalance reactions that we found: 9


In [22]:
print(sorted(list(im_reaction_set)))

[241, 289, 368, 402, 482, 921, 928, 983, 1014]


Organism:

Version: 

Year:

Paper link: 

Model Link: 

In [62]:
import gc
gc.collect()

0

23